In [1]:
%cd ../

D:\Source\tbdx23


In [2]:
import os
import pandas as pd
import numpy as np
from scipy.optimize import minimize_scalar
from sim.core.cascade import RepoCascade
from sim.core.inputs import load_inputs
from sim.zaf.dy import ModelZAF

In [3]:
iso = 'ZAF'

in_folder = f'results/A_{iso}'
out_folder = f'results/B_{iso}'

In [4]:
os.makedirs(out_folder, exist_ok=True)

In [5]:
pars_cs = pd.read_csv(f'results/pars_{iso}.csv')
repo_cs = RepoCascade(pars_cs, 2010)

inp = load_inputs(f'data/pars/{iso}')
model = ModelZAF(inp)

In [6]:
tarq = pd.read_csv(f'data/pars/{iso}/targets_q.csv').set_index('Time')
tarq

,iso,Country,Year,q,QCNR
Time,,,,,
2020.125,ZAF,South Africa,2020,1,0.001061
2020.375,ZAF,South Africa,2020,2,0.000647
2020.625,ZAF,South Africa,2020,3,0.000689
2020.875,ZAF,South Africa,2020,4,0.000809
2021.125,ZAF,South Africa,2021,1,0.000918
2021.375,ZAF,South Africa,2021,2,0.000556
2021.625,ZAF,South Africa,2021,3,0.000598
2021.875,ZAF,South Africa,2021,4,0.000720


In [7]:
particles = pd.read_csv(f'{in_folder}/Post.csv')
particles = [dict(row) for _, row in particles.iterrows()]
for p in particles:
    p['cas'] = repo_cs.sample()

In [8]:
ts = list(tarq.index)
ts += [ts[-1] + 1 / 4]
ts = np.array(ts)

In [9]:
sims = list()
for p in particles:
    p['cas'] = repo_cs.sample()
    
    ys, _, _ = model.simulate_to_preCOVID(p)
    sims.append((p, ys))


In [10]:
def fn(k, sims, tarq):
    ts = list(tarq.index)
    ts += [ts[-1] + 1 / 4]
    ts = np.array(ts)
    
    ds = list()
    
    for p, ys in sims:
        p['k_covid'] = k
        _, ms, _ = model.simulate_to_postCOVID(p, ys, t_eval=ts)
        d = (tarq.QCNR - ms.CNR).pow(2).sum()
        ds.append(d)
        
    return sum(ds)


In [11]:
opt = minimize_scalar(fn, bounds=[0.1, 1], args=(sims, tarq, ), method='bounded')

In [12]:
opt

     fun: 0.00010940476512064838
 message: 'Solution found.'
    nfev: 21
  status: 0
 success: True
       x: 0.9977063502687685

In [13]:
k = opt.x
mss = list()

for i, (p, ys) in enumerate(sims):
    p['k_covid'] = k
    _, ms, _ = model.simulate_to_postCOVID(p, ys, t_eval=ts)
    mss.append(ms.assign(Key = i))
    
mss = pd.concat(mss)

In [14]:
mss.to_csv(f'{out_folder}/RunPost.csv')

In [15]:
pss = pd.DataFrame([p for p, _ in sims])
pss.to_csv(f'{out_folder}/Post.csv')